# Local Model Evaluation

### Used for testing the performance of a model locally

In [1]:
# Imports
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from tensorflow import uint8, float32, cast
from os import path, walk
from numpy import argmax, argsort, fliplr
from pandas import Series

In [2]:
# Constants
TEST_DIR = path.join('Fresh', 'test')
MODEL_DIR = path.join('model_dir', 'Xception', 'transfer-learning', 'best3', 'runtime')
K = 1

In [3]:
# Get the names of the classes
class_names = []
for subdir, dirs, files in walk(TEST_DIR):
    try:
        class_names.append(subdir.split('\\')[2])
    except:
        pass
num_classes = len(class_names)

In [4]:
# Setup test data generator
test_data_gen = ImageDataGenerator()
test_generator = test_data_gen.flow_from_directory(TEST_DIR,
                                                     target_size=(299,299),
                                                     color_mode='rgb',
                                                     batch_size=16,
                                                     class_mode='categorical',
                                                     shuffle=False)

Found 654 images belonging to 34 classes.


In [5]:
# Load the model
runtime_model = load_model(MODEL_DIR)
runtime_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 299, 299, 3)]     0         
_________________________________________________________________
tf_op_layer_Cast_1 (TensorFl (None, 299, 299, 3)       0         
_________________________________________________________________
tf_op_layer_RealDiv_1 (Tenso (None, 299, 299, 3)       0         
_________________________________________________________________
tf_op_layer_Sub_1 (TensorFlo (None, 299, 299, 3)       0         
_________________________________________________________________
sequential (Sequential)      (None, 34)                21452458  
Total params: 21,452,458
Trainable params: 21,397,930
Non-trainable params: 54,528
_________________________________________________________________


In [6]:
# Get the predictions (this one takes awhile)
predictions = runtime_model.predict(test_generator)
predictions

array([[8.2666492e-01, 1.9856320e-04, 2.4131670e-09, ..., 1.0629472e-03,
        6.6293133e-03, 3.6272431e-05],
       [5.3688701e-02, 1.0768525e-04, 4.2657553e-07, ..., 6.0404191e-06,
        5.5576231e-05, 8.9321856e-04],
       [1.4287499e-01, 2.2377526e-05, 1.0971102e-05, ..., 3.4897273e-06,
        3.6160382e-05, 1.0658685e-03],
       ...,
       [2.5620317e-05, 2.6101959e-06, 5.4917598e-07, ..., 1.1797726e-04,
        2.3390373e-06, 1.5990908e-01],
       [1.8151147e-06, 2.5053848e-09, 6.2008869e-09, ..., 4.4727722e-07,
        7.9848589e-10, 9.7655129e-01],
       [5.4601196e-04, 1.0539003e-06, 1.3353160e-05, ..., 3.5073172e-04,
        3.7012726e-06, 1.5102512e-01]], dtype=float32)

In [7]:
# Function to get an array of the top k class names from the predictions array
# for each prediction
def get_top_k_class_predictions(predictions, k, class_names):
    sorted_indices = argsort(predictions, axis=1)
    flipped = fliplr(sorted_indices)
    top_k_indices = flipped[:, :k]
    res = []
    for row in top_k_indices:
        r = []
        for elem in row:
            r.append(class_names[elem])
        res.append(r)
    return res

In [8]:
# Convert to names
prediction_names = get_top_k_class_predictions(predictions, K, class_names)
file_names = test_generator.filenames
correctness_lst = [(file_name, pred_name) for file_name, pred_name in zip(file_names, prediction_names)]
prediction_names

[['Apple'],
 ['Peach'],
 ['Mango'],
 ['Apple'],
 ['Apple'],
 ['Apple'],
 ['Mango'],
 ['Apple'],
 ['Peach'],
 ['Peach'],
 ['Blackberries'],
 ['Peach'],
 ['Blackberries'],
 ['Blackberries'],
 ['Blackberries'],
 ['Apple'],
 ['Apple'],
 ['Apple'],
 ['Apple'],
 ['Mango'],
 ['Mango'],
 ['Tomato'],
 ['Blackberries'],
 ['Apple'],
 ['Mango'],
 ['Blackberries'],
 ['Peach'],
 ['Mango'],
 ['Apple'],
 ['Apple'],
 ['Apple'],
 ['Melon'],
 ['Avocado'],
 ['Avocado'],
 ['Avocado'],
 ['Avocado'],
 ['Avocado'],
 ['Avocado'],
 ['Avocado'],
 ['Avocado'],
 ['Avocado'],
 ['Avocado'],
 ['Melon'],
 ['Tomato'],
 ['Melon'],
 ['Avocado'],
 ['Avocado'],
 ['Avocado'],
 ['Avocado'],
 ['Avocado'],
 ['Avocado'],
 ['Avocado'],
 ['Avocado'],
 ['Melon'],
 ['Avocado'],
 ['Avocado'],
 ['Avocado'],
 ['Melon'],
 ['Melon'],
 ['Melon'],
 ['Melon'],
 ['Avocado'],
 ['Avocado'],
 ['Avocado-cut'],
 ['Avocado-cut'],
 ['Avocado-cut'],
 ['Avocado-cut'],
 ['Avocado-cut'],
 ['Avocado-cut'],
 ['Avocado-cut'],
 ['Avocado-cut'],
 ['Avocado

In [9]:
# Given the list of file_names, top_k_predictions, returns a Pd.Series
# to present accuracy results
def get_top_k_accuracy(correctness_lst):
    corrects = {}
    totals = {}
    for file_name, top_k in correctness_lst:
        class_name = file_name.split('\\')[0]
        if class_name not in corrects:
            corrects[class_name] = 0
            totals[class_name] = 0
        totals[class_name] += 1
        if class_name in top_k:
            corrects[class_name] += 1
    accuracy_dict = {c_key : corrects[c_key] / totals[t_key] for c_key, t_key in zip(corrects, totals)}
    accuracy_series = ((Series(accuracy_dict)*100).round(1)).astype(str) + '%'
    accuracy_series2 = (Series(accuracy_dict).round(2)*100).astype(str) + '%'
    total_amt = sum(totals[t_key] for t_key in totals)
    correctness_amt = sum(corrects[c_key] for c_key in corrects)
    spacer_series = Series({'-----':'--------'})
    accuracy_series = accuracy_series.append(spacer_series)
    total_series = (Series({'Total' : correctness_amt / total_amt})*100).round(1).astype(str) + '%'
    all_series = accuracy_series.append(total_series)
    return all_series

print(get_top_k_accuracy(correctness_lst))

Apple                 41.9%
Avocado               71.9%
Avocado-cut           93.5%
Banana               100.0%
Bell Pepper           93.1%
Blackberries         100.0%
Blueberries           94.4%
Cherries              48.1%
Coconut               50.0%
Cucumber              96.0%
Cucumber-cut         100.0%
Grapes                95.0%
Kiwi                   0.0%
Kiwi-cut              93.3%
Lemon                 83.3%
Lime                  50.0%
Lychee                36.4%
Mango                 67.7%
Melon                 96.2%
Orange                72.0%
Orange-cut            86.2%
Papaya                54.5%
Papaya-cut           100.0%
Peach                 53.3%
Pear                  73.5%
Pineapple            100.0%
Plum                  16.7%
Pomegranate           35.3%
Pomegranate-cut      100.0%
Raspberries          100.0%
Strawberries          79.3%
Tomato               100.0%
Watermelon           100.0%
Watermelon-cut        66.7%
-----              --------
Total               